# Rats Motion Mapper

#### import modules
wavelet : run the wavelet transform;

run_tsne : run tSNE;

density_map : apply a gaussian convolution to the scatter points in the behavioral space;

embedding_z : re-embedding using tSNE

In [ ]:
from wavelet import *
from run_tsne import *
from density_map import *
from embedding_z import *

In [ ]:
import seaborn as sns
import multiprocessing as mp
from math import factorial
import itertools
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import scipy as sp
import matplotlib.gridspec as grd
from scipy.stats import gaussian_kde
%matplotlib notebook

### Import data

1. The data come from Bence Olveczky and Jesse Marshall's lab

2. The raw data set contains the 3D positions of 20 markers on rats over a time period of 21 days at a sampling rate of 300 Hz, thus a 22,902,319 $\times$ 60 matrix. 

3. The twenty markers are labeled as HeadF, HeadB, HeadL, SpineF, SpineM, SpineL, Offset1, Offset2, HipL, HipR, ElbowL, ArmL, ShoulderL, ShoulderR, ElbowR, ArmR, KneeR, KneeL, ShinL, ShinR. 

4. A short version data is used here (300,000 consecutive time points extracted from the raw data).

In [ ]:
filepath = "data/rat_data_short.txt"
data = np.loadtxt(filepath)

### Translation of data
Translating the position values into segment lengths and joint angles to filter out variance. Using

$$ ||L|| =  [\sum_{i = x,y,z} abs(i_{1}-i_{2})^2 ]^{1/2}$$
$$ \Theta = arccos(\frac{\vec{u}-\vec{v}}{||\vec{u}||\cdot||\vec{v}||})$$


In [ ]:
# parse data, find segment lengths and joint angles
HeadF = data[:,0:3]
HeadB = data[:,3:6]
HeadL = data[:,6:9]
SpineF = data[:,9:12]
SpineM = data[:,12:15]
SpineL = data[:,15:18]
Offset1 = data[:,18:21]
Offset2 = data[:,21:24]
HipL = data[:,24:27]
HipR = data[:,27:30]
ElbowL = data[:,30:33]
ArmL = data[:, 33:36]
ShoulderL = data[:, 36:39]
ShoulderR = data[:, 39:42]
ElbowR = data[:, 42:45]
ArmR = data[:,45:48]
KneeR = data[:,48:51]
KneeL = data[:,51:54]
ShinL = data[:, 54:57]
ShinR = data[:, 57:60]

def joint_angle(x1 = np.array([]), x2 = np.array([]), x3 = np.array([])):
# x1, x2, x3 are your first, second, third points
    angle = np.zeros(x1.shape[0])
    for i in range(x1.shape[0]):
        angle[i] = np.arccos(np.dot((x1[i]-x2[i]), (x3[i]-x2[i])) / (np.linalg.norm(x1[i]-x2[i]) * np.linalg.norm(x3[i]-x2[i])))            
    return angle

def calculate_combinations(n, r):
    return factorial(n) // factorial(r) // factorial(n-r)

def parse_body_parts(body_part):
# a body part contains all relevant tracking data for a certain body part
# e.g. for Head, is HeadF, HeadB, HeadL and SpineF
    count1 = 0
    count2 = 0
    Segs = np.zeros((calculate_combinations(len(body_part),2),np.shape(body_part[0])[0]))
    Joint_angles = np.zeros((calculate_combinations(len(body_part),3),np.shape(body_part[0])[0])) 
    for ele in itertools.combinations(body_part,2):
        Segs[count1] = np.linalg.norm(ele[0]-ele[1], axis = 1)
        count1 += 1
    for ele in itertools.combinations(body_part,3):
        Joint_angles[count2] = joint_angle(ele[0], ele[1], ele[2])
        count2 += 1
    return Segs,Joint_angles

Head_seg,Head_angle = parse_body_parts((HeadF,HeadB,HeadL,SpineF))
print('Done HEAD segment lengths and joint angles')
Body_seg,Body_angle = parse_body_parts((SpineF,SpineM,SpineL,Offset1,Offset2))
print('Done BODY segment lengths and joint angles')
LimbL_seg,LimbL_angle = parse_body_parts((ElbowL,ArmL,ShoulderL,SpineF))
print('Done LIMBL segment lengths and joint angles')
LimbR_seg,LimbR_angle = parse_body_parts((ElbowR,ArmR,ShoulderR,SpineF))
print('Done LIMBR segment lengths and joint angles')
LegL_seg,LegL_angle = parse_body_parts((HipL,KneeL,ShinL,SpineL))
print('Done LEGL segment lengths and joint angles')
LegR_seg,LegR_angle = parse_body_parts((HipR,KneeR,ShinR,SpineL))
print('Done LEGR segment lengths and joint angles')

#### Plots of segment lengths and joint angles

<img src="./plots/joint_angles.png" /><img src="./plots/seg_lengths.png" />


In [ ]:
# combine all data in one data set
All_seg_angle = np.zeros((Head_seg.shape[0]+Head_angle.shape[0]+
                          Body_seg.shape[0]+Body_angle.shape[0]+LimbL_seg.shape[0]+LimbL_angle.shape[0]+
                          LimbR_seg.shape[0]+LimbR_angle.shape[0]+LegL_seg.shape[0]+LegL_angle.shape[0]+
                          LegR_seg.shape[0]+LegR_angle.shape[0], data.shape[0]))
All_seg_angle[:Head_seg.shape[0]] = Head_seg
temp = Head_seg.shape[0]
All_seg_angle[temp:temp+Head_angle.shape[0]] = Head_angle
temp += Head_angle.shape[0]
All_seg_angle[temp:temp+Body_seg.shape[0]] = Body_seg
temp += Body_seg.shape[0]
All_seg_angle[temp:temp+Body_angle.shape[0]] = Body_angle
temp += Body_angle.shape[0]
All_seg_angle[temp:temp+LimbL_seg.shape[0]] = LimbL_seg
temp += LimbL_seg.shape[0]
All_seg_angle[temp:temp+LimbL_angle.shape[0]] = LimbL_angle
temp += LimbL_angle.shape[0]
All_seg_angle[temp:temp+LimbR_seg.shape[0]] = LimbR_seg
temp += LimbR_seg.shape[0]
All_seg_angle[temp:temp+LimbR_angle.shape[0]] = LimbR_angle
temp += LimbR_angle.shape[0]
All_seg_angle[temp:temp+LegL_seg.shape[0]] = LegL_seg
temp += LegL_seg.shape[0]
All_seg_angle[temp:temp+LegL_angle.shape[0]] = LegL_angle
temp += LegL_angle.shape[0]
All_seg_angle[temp:temp+LegR_seg.shape[0]] = LegR_seg
temp += LegR_seg.shape[0]
All_seg_angle[temp:temp+LegR_angle.shape[0]] = LegR_angle


## The wavelet transform
The morlet continuous wavelet transform is implemented to measure the power across a range of frequencies for each variable. Unlike Fourier spectrogram, where the time window has a fixed size, the wavelet transform strikes a balance between time domain and frequency domain, where we can see low frequency components at long time windows and high frequency resolution and vice versa for high frequency components.

#### Running the wavelet transform:

The resulted power spectrum $S(f,k;t)$ possess 75 dyadically spaced frequency channels (1 - 150Hz) for each variable.

In [ ]:
# Running Wavelet Transform
numPeriods = 75 #set number of period
omega0 = 5
samplingFreq = 300.
dt = 1/samplingFreq
# set frequency 
minF = 1.  
maxF = 150.
minT = 1/maxF; maxT = 1/minF;
Ts = []; f = []
for i in range(numPeriods):
    Ts.append(minT*2**(i*np.log(maxT/minT)/(np.log(2)*(numPeriods-1))))
f = [1/i for i in Ts]
f.reverse()

numModes = All_seg_angle.shape[0]   #running wavelet transform on first 20 eigenmodes
N = np.shape(All_seg_angle)[1]  #number of time points
amp = np.zeros((N, numModes*numPeriods))  

# multiprocess 
pool = mp.Pool(processes=18)
results = pool.map(fastWavelet_morlet_convolution_parallel,
                   itertools.izip(itertools.chain(All_seg_angle[:numModes]),
                                  itertools.repeat(f), itertools.repeat(omega0), itertools.repeat(dt),itertools.count(1)))
pool.close()
pool.join()
              
for i, result in zip(range(numModes), results):
    amp[:,(i)*numPeriods:(i+1)*numPeriods] = np.transpose(result)
    
where_are_NaNs = np.isnan(amp)
where_are_Infs = np.isinf(amp)
amp[where_are_NaNs] = 1e-12
amp[where_are_Infs] = 1e-12


#### Plot the spectrogram:

In [ ]:
fig = plt.figure()
time = [i * dt for i in range(N)]
T, F = np.meshgrid(time,f)

gs = grd.GridSpec(3, 2,  width_ratios=[20,1], wspace=0.1,hspace=0.6)

# image plot
ax1 = plt.subplot(gs[0])
a = np.transpose(amp[:,(0)*numPeriods:numPeriods+(0)*numPeriods])
im = ax1.pcolormesh(T,F,a,cmap = 'CMRmap_r', norm=matplotlib.colors.LogNorm(vmin=a.min(), vmax=a.max()))
plt.title('wavelet transform of segment length HeadF-HeadB')
plt.ylabel('frequency')
ax1.set_yscale('log')

colorAx = plt.subplot(gs[1])
cb = plt.colorbar(im, cax = colorAx)
cb.set_label('RWU')

ax2 = plt.subplot(gs[2])
a = np.transpose(amp[:,(6)*numPeriods:numPeriods+(6)*numPeriods])
im = ax2.pcolormesh(T,F,a,cmap = 'CMRmap_r', norm=matplotlib.colors.LogNorm(vmin=a.min(), vmax=a.max()))
plt.title('wavelet transform of joint angle HeadF-HeadB-HeadL')
plt.ylabel('frequency')
ax2.set_yscale('log')

colorAx = plt.subplot(gs[3])
cb = plt.colorbar(im, cax = colorAx)
cb.set_label('RWU')

ax2 = plt.subplot(gs[4])
a = np.transpose(amp[:,(10)*numPeriods:numPeriods+(10)*numPeriods])
im = ax2.pcolormesh(T,F,a,cmap = 'CMRmap_r', norm=matplotlib.colors.LogNorm(vmin=a.min(), vmax=a.max()))
plt.title('wavelet transform of segment length SpineF-SpineM')
plt.xlabel('time (s)')
plt.ylabel('frequency')
ax2.set_yscale('log')

colorAx = plt.subplot(gs[5])
cb = plt.colorbar(im, cax = colorAx)
cb.set_label('RWU')

plt.savefig('./plots/wavelet.png')
plt.show()


#### Plots of spectrogram
<img src="./plots/wavelet.png" />

## tSNE

In our implementation, the input $X$ consists of 30,000 randomly sampled feature vectors as training set, so a 30,000 $\times$ 5250 matrix, $X =\{x_{1},x_{2},...,x_{30,000}\}$. In tSNE, the conserved invariants of high dimensional input is a joint probability distribution $P$ over all pairs of non-identical points, with entries
\begin{equation}
    p_{ij} = \frac{exp(-\Delta^2_{ij}/\sigma)}{\sum_k\sum_{l\neq k} exp(-\Delta^2_{kl}/\sigma)},
\end{equation}
where $\Delta^2_{ij}$ is the pairwise distance in euclidean space.

For the output 2D map, denoted by $Y = \{y_1,y_2,...,y_{30,000}\}$, we aim to model it in such a way that its joint probability distribution $q_{ij}$ is as close as possible to $p_{ij}$. A reasonable measurement of the similarity is the Kullback-Leibler divergence between P and Q:
\begin{equation}
    C(Y) = KL(P||Q) =\sum_i\sum_{j\neq i}p_{ij}log\frac{p_{ij}}{q_{ij}}.
\end{equation}
We use a gradient descent algorithm to minimize $C(Y)$ as it is almost always non-convex.

Now we have to define the joint probabilities $q_{ij}$ in the low-dimensional map. The key property of tSNE is that in the low-dimensional map the similarity between two points is proportional to a Student-t distribution:
\begin{equation}
    q_{ij} = \frac{(1+||y_i-y_j||^2)^{-1}}{\sum_k\sum_{l\neq k}(1+||y_k-y_l||^2)^{-1}}.
\end{equation}

In [ ]:
training_set = 30000 #set size of training set
batch_index = np.random.randint(300000, size = training_set)
#np.savetxt('./data/training_batch_index_head.txt', batch_index)
motion_map = np.zeros((30000,2))

training_batch = amp[batch_index,:]
results = tsne(training_batch, 2,perplexity = 30)
motion_map = results
np.savetxt('./data/motion_map_head.txt', results)

#### Scatter plot of the 2D behavioral space

In [ ]:
from scipy.stats import gaussian_kde
x = motion_map_head[:,0]
y = motion_map_head[:,1]
xy = np.vstack([x,y])
z = gaussian_kde(xy)(xy)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(x,y, c=z, s=1, edgecolor='')
plt.xlim([motion_map.min(),motion_map.max()])
plt.ylim([motion_map.min(),motion_map.max()])
plt.title('scatter plot of training set')
ax.set_aspect('auto')
plt.savefig('./plots/motion_map_scatter.png')

<img src="./plots/motion_map_scatter.png" />

#### Density plot

Apply a Gaussian convolution to each point in the scatter map to obtain the density plots

In [ ]:
plot_density_map(motion_map, beta = 50,resolution = 1000)

<img src="./plots/motion_map.png" />

## Re-embedding the remaining points


For a single point $Z$, the transition probability with the training set $p_{j|z}$ and transition probability in the embedded space $q_{j|\zeta}$ are given by

\begin{align}
     p_{j|z} & = \frac{exp(-\Delta^2_{jz}/\sigma_z)}{\sum_{k\in \text{training set}} exp(-\Delta^2_{zk}/\sigma_z)} \\
     q_{j|\zeta} &= \frac{(1+||y_\zeta-y_j||^2)^{-1}}{\sum_{k'\in \text{embedding space}}(1+||y_\zeta-y_{k'}||^2)^{-1}}.
\end{align}

Similarly, for each $z$, we seek the $\zeta$ that minimizes the cost function

\begin{equation}
    C(z) = KL(p_{x|z}||q_{y|\zeta}) =\sum_{x\in \text{training set}}p_{x|z}log\frac{p_{x|z}}{q_{y(x)|\zeta}}
\end{equation}



#### Re-embedding the time points 0 - 20 s.

In [ ]:
embedding_values = embedding_Z (amp[:6000], amp[batch_index,:], motion_map, 1000)
np.savetxt('./data/re_embedding.txt', embedding_values)

#### Plot the re-embedding values $z_1$,  $z_2$:
Dynamics in the 2D behavioral space

In [ ]:
fig = plt.figure(figsize = (10,3))

plt.plot(np.linspace(0,20,6000),embedding_values[:,0], label = 'z_1', linewidth = 0.5)
plt.plot(np.linspace(0,20,6000),embedding_values[:,1], label = 'z_2', linewidth = 0.5)
plt.title('two dimension re-embedding values')
plt.xlabel('time (s)')
plt.legend()
plt.savefig('./plots/re-embedding.png')
plt.show()

<img src="./plots/re-embedding.png" />

The velocity profiles of the re-embedding values feature a distribution of two-component log-Gaussian mixture model.

In [ ]:
from scipy.stats import lognorm

two_peak_velocity = np.zeros(5999)

for i in range(5999):
    velocity = np.sqrt((embedding_values[i+1,0]-embedding_values[i,0])**2 + (embedding_values[i+1,1]-embedding_values[i,1])**2)
    two_peak_velocity[i] = velocity
# plot histogram in log space
ax = plt.subplot(111)
y,x,_ = ax.hist(two_peak_velocity, bins=np.logspace(-2,3,200),label = 'pdf')
ax.set_xscale("log")
x=(x[1:]+x[:-1])/2

def log_gauss(x,mu,sigma,A):
    return A*np.exp(-(np.log(x)-mu)**2/2/sigma**2)

def bimodal(x,mu1,sigma1,A1,mu2,sigma2,A2):
    return log_gauss(x,mu1,sigma1,A1)+log_gauss(x,mu2,sigma2,A2)

(mu1,sigma1,A1,mu2,sigma2,A2),pcov = sp.optimize.curve_fit(bimodal,x,y)
pdf = bimodal(x,mu1,sigma1,A1,mu2,sigma2,A2)
ax.plot(x, pdf, 'r', label ='fitted  bimodal log_Gaussian')
plt.xlabel('velocity')
plt.legend()
plt.savefig('./plots/two_peak.png')
plt.show()

<img src="./plots/two_peak.png" />